In [ ]:
import random
import copy

In [21]:
def count_attacking_queens(state):
    attacking_queens = 0
    for col, row in enumerate(state):
        for other_col, other_row in enumerate(state):
            if col == other_col: continue
            if row == other_row: attacking_queens += 1 # Same row
            if (other_row + other_col) == (col + row): attacking_queens += 1 # Same diagonal => \
            if (row - col) == (other_row - other_col): attacking_queens += 1 # Same diagonal => /
    return attacking_queens / 2

# Count attacked pair of queens of Fig 4.3.b
print(count_attacking_queens([3,2,1,4,3,2,1,2]))

17.0


In [22]:
class Action:
    def __init__(self, col, new_row):
        self.col = col
        self.new_row = new_row

In [23]:
class Node:
    def __init__(self, state, parent, action, path_cost):
        self.state = state
        self.parent = parent
        self.action = action 
        self.path_cost = path_cost
    
    # Number of attacking pairs
    @property
    def value(self):
        return -1 * count_attacking_queens(self.state)
    

def expand(problem, node):
    s = node.state
    child_nodes = []
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        child_nodes.append(Node(s1, node, action, cost))
    return child_nodes

In [24]:
class EightQueensProblem():
    def __init__(self, initial_state): 
        self.initial_state = initial_state

    def actions(self, state):
        all_actions = []
        for col, queen in enumerate(state):
            for new_row in range(8):
                if new_row == queen: continue
                all_actions.append(Action(col, new_row))
        return all_actions
    
    # Transition model
    def result(self, state, action):
        new_state = copy.deepcopy(state)
        new_state[action.col] = action.new_row
        return new_state
    
    # A set of one or more goals, in this problem there is only one goal state
    def is_goal(self, state):
        for i, queen in enumerate(state):
            for j, other_queen in enumerate(state):
                if j == i: continue
                if other_queen == queen: return False # Same row
                if other_queen + j == queen + i: return False # Same diagonal => \
                if j - other_queen == i - other_queen: return  False # Same diagonal => /
        return True
    
    def action_cost(self, s, a, s1): return 1

    def find_best_neighbor(self, nodes: list[Node]):
        random.shuffle(nodes)
        best_node = nodes[0]
        for node in nodes:
            if node.value > best_node.value:
                best_node = node
        return best_node
        

In [25]:
def hill_climbing(problem: EightQueensProblem):
    current = Node(problem.initial_state, None, None, 0)
    steps = 0
    while True:
        neighbors = expand(problem, current)
        # No neighbors
        if not len(neighbors): break

        # Max neighbor
        neighbor = problem.find_best_neighbor(neighbors)
        
        # No more higher value neighbor (at local max, or global max)
        if neighbor.value <= current.value: break

        # Continue walking
        current = neighbor
        steps += 1
    
    return current.state, steps


In [26]:
def solve_random_board_hill_climbing(num_of_board):
    succeeded_records = []
    failed_records = []
    for i in range(num_of_board):
        p = EightQueensProblem([random.randint(0, 7) for i in range(8)])
        sol, steps = hill_climbing(p)
        if count_attacking_queens(sol) == 0: 
            succeeded_records.append(steps)
        else:
            failed_records.append(steps)
    print("------Hill climbing---------")
    print('- Average steps to succeed:', sum(succeeded_records) / len(succeeded_records))
    print('- Average steps before failure:', sum(failed_records) / len(failed_records))
    print('- Number of success:', len(succeeded_records), f'{len(succeeded_records)/num_of_board * 100}%')
    print('- Number of failure:', len(failed_records), f'{len(failed_records)/num_of_board * 100}%')

solve_random_board_hill_climbing(1000)

------Hill climbing---------
- Average steps to succeed: 4.121212121212121
- Average steps before failure: 3.055299539170507
- Number of success: 132 13.200000000000001%
- Number of failure: 868 86.8%


In [28]:
def hill_climbing_with_sideway_move(problem: EightQueensProblem):
    current = Node(problem.initial_state, parent=None, action=None, path_cost=0)
    steps = 1
    consecutive_limit = 99
    while True:
        neighbors = expand(problem, current)
        # No neighbors
        if not len(neighbors): break
        
        # Max neighbor
        neighbor = problem.find_best_neighbor(neighbors)
        
        if neighbor.value < current.value: break
        
        # No better value neighbors (hill-climbing is at the plateau)
        if neighbor.value == current.value:
            if consecutive_limit <= 0: break
            else: consecutive_limit -= 1
        
        # Continue walking
        current = neighbor
        steps += 1
    return current.state, steps

In [29]:
def solve_random_board_hill_climbing_with_sideway(num_of_board):
    succeeded_records = []
    failed_records = []
    for i in range(num_of_board):
        p = EightQueensProblem([random.randint(0, 7) for i in range(8)])
        sol, steps = hill_climbing_with_sideway_move(p)
        if count_attacking_queens(sol) == 0: 
            succeeded_records.append(steps)
        else:
            failed_records.append(steps)

    print("------Hill climbing with side way move (limit 100)--------")
    print('- Average steps to succeed: ', sum(succeeded_records) / len(succeeded_records))
    print('- Average steps before failure: ', sum(failed_records) / len(failed_records))
    print('- Number of success: ', len(succeeded_records), f'{len(succeeded_records)/num_of_board * 100}%')
    print('- Number of failure: ', len(failed_records), f'{len(failed_records)/num_of_board * 100}%')

solve_random_board_hill_climbing_with_sideway(1000)

------Hill climbing with side way move (limit 100)--------
- Average steps to succeed:  20.192680301399353
- Average steps before failure:  66.25352112676056
- Number of success:  929 92.9%
- Number of failure:  71 7.1%
